In [75]:
# ------------------------------------------------------- 
# Assignment (include number)
# Written by (include your name and student id)
# For COMP 472 Section (your lab section) – Summer 2020
# --------------------------------------------------------

In [76]:
import pandas as pd
import math
import re
import sys
import matplotlib.pyplot as plt 
import os
import numpy as np
import string 

# Task 1: Extract the data and build the model

In [77]:
df = pd.read_csv('dataset.csv')
df = df[["Title","Post Type","year"]]

df_train = df.loc[df['year'] == 2018]
df_test = df.loc[df['year'] == 2019]

In [78]:
story_dictionary = {}      # stores words in story class and their frequencies
ask_hn_dictionary = {}     # stores words in ask_hn class and their frequencies
show_hn_dictionary = {}    # stores words in show_hn class and their frequencies
poll_dictionary = {}       # stores words in poll class and their frequencies
vocabulary = set()         # stores all unique words present in dataset
word_not_count = set()     # stores all unique removed words present in dataset
stopword = set()
story_cp_dictionary = {}   # stores words in story class and their conditional probabilities
ask_hn_cp_dictionary = {}  # stores words in ask_hn class and their conditional probabilities
show_hn_cp_dictionary = {} # stores words in show_hn class and their conditional probabilities
poll_cp_dictionary = {}    # stores words in poll class and their conditional probabilities

In [79]:
def data_clear():
    
    global story_dictionary      # stores words in story class and their frequencies
    global ask_hn_dictionary      # stores words in ask_hn class and their frequencies
    global show_hn_dictionary    # stores words in show_hn class and their frequencies
    global poll_dictionary        # stores words in poll class and their frequencies
    global story_cp_dictionary    # stores words in story class and their conditional probabilities
    global ask_hn_cp_dictionary   # stores words in ask_hn class and their conditional probabilities
    global show_hn_cp_dictionary # stores words in show_hn class and their conditional probabilities
    global poll_cp_dictionary     # stores words in poll class and their conditional probabilities
    global vocabulary         # stores all unique words present in dataset
    global word_not_count     # stores all unique removed words present in dataset
    global stopword 

    story_dictionary = {}      # stores words in story class and their frequencies
    ask_hn_dictionary = {}     # stores words in ask_hn class and their frequencies
    show_hn_dictionary = {}    # stores words in show_hn class and their frequencies
    poll_dictionary = {}       # stores words in poll class and their frequencies
    vocabulary = set()         # stores all unique words present in dataset
    word_not_count = set()     # stores all unique removed words present in dataset
    stopword = set()
    story_cp_dictionary = {}   # stores words in story class and their conditional probabilities
    ask_hn_cp_dictionary = {}  # stores words in ask_hn class and their conditional probabilities
    show_hn_cp_dictionary = {} # stores words in show_hn class and their conditional probabilities
    poll_cp_dictionary = {}    # stores words in poll class and their conditional probabilities
    
    # story_dictionary.clear()
    # ask_hn_dictionary.clear()
    # show_hn_dictionary.clear()
    # poll_dictionary.clear()
    # story_cp_dictionary.clear()
    # ask_hn_cp_dictionary.clear()
    # show_hn_cp_dictionary.clear()
    # poll_cp_dictionary.clear()
    # vocabulary.clear()
    # word_not_count.clear()
    # stopword.clear()
    


In [80]:
'''
dictCheck(word)  =  if this word is not present in any other dictionary, add it with count 0
word = All words after removing invalid words 

'''

def dictCheck(word):
    
    global story_dictionary      # stores words in story class and their frequencies
    global ask_hn_dictionary      # stores words in ask_hn class and their frequencies
    global show_hn_dictionary    # stores words in show_hn class and their frequencies
    global poll_dictionary        # stores words in poll class and their frequencies

    
    if word not in story_dictionary:
        story_dictionary[word] = 0
    if word not in ask_hn_dictionary:
        ask_hn_dictionary[word] = 0
    if word not in show_hn_dictionary:
        show_hn_dictionary[word] = 0
    if word not in poll_dictionary:
        poll_dictionary[word] = 0

In [81]:
'''
word_count_directory(df_train)  = build vocabulary from training set
df_train = dataframe of training set (year=2018)

'''

def word_count_directory(df_train,ch):
    
    global story_dictionary      # stores words in story class and their frequencies
    global ask_hn_dictionary      # stores words in ask_hn class and their frequencies
    global show_hn_dictionary    # stores words in show_hn class and their frequencies
    global poll_dictionary        # stores words in poll class and their frequencies
    global vocabulary         # stores all unique words present in dataset
    global word_not_count     # stores all unique removed words present in dataset
    global stopword 
    print(type(vocabulary))

    # intialize no of story, ask_hn, show_hn and poll class 
    no_of_story_class = len(df_train.loc[df['Post Type'] == "story"])
    no_of_ask_hn_class = len(df_train.loc[df['Post Type'] == "ask_hn"])
    no_of_show_hn_class = len(df_train.loc[df['Post Type'] == "show_hn"])
    no_of_poll_class = len(df_train.loc[df['Post Type'] == "poll"])
    
    if ch == 1:
        with open('stopwords.txt',encoding='latin-1') as infile:
            for line1 in infile:
                line1 = line1.strip()    # Remove the leading spaces and newline character
                lower_line1 = str.lower(line1)
                stopword.add(lower_line1)
                
    for index, row in df_train.iterrows():
        class_type = row['Post Type']
        line = row['Title']
        line = line.strip()             # Remove the leading spaces and newline character
        lower_line = str.lower(line)    # Convert characters in line to lowercase to avoid case mismatch
        
        valid_words = re.split("[^a-zA-Z]",lower_line)       # filter words following the given regex
        valid_words = list(filter(None, valid_words))        # filter words with length greater than 0
        
        res = re.sub('['+string.punctuation+'–‘«“”"’\']', ' ', lower_line).split()  # remove all special character ans numbers
        res = list(filter(None, res))
        invalid_words  = list(set(res) - set(valid_words))   # diff. between valid & invalid words
        for w in invalid_words:
            word_not_count.add(w)
        
        
        for word in valid_words:
            if ch == 1:
                if word in stopword:
                    word_not_count.add(word)
                    continue
                
            
            # check for class type
            if class_type == 'story':
                
                # Check if the word is already in dictionary
                if word in story_dictionary:
                    story_dictionary[word] += 1
                else:
                    story_dictionary[word] = 1     # add word to dictionary with count 1
                    vocabulary.add(word)           # add word to vocabulary set
                    
            elif class_type == 'ask_hn':

                if word in ask_hn_dictionary:
                    ask_hn_dictionary[word] += 1
                else:
                    ask_hn_dictionary[word] = 1   
                    vocabulary.add(word)
                                
            elif class_type == 'show_hn':

                if word in show_hn_dictionary:
                    show_hn_dictionary[word] += 1
                else:
                    show_hn_dictionary[word] = 1  
                    vocabulary.add(word)
                                        
            elif class_type == 'poll':

                if word in poll_dictionary:
                    poll_dictionary[word] += 1
                else:
                    poll_dictionary[word] = 1   
                    vocabulary.add(word)
         
           # if this word is not present in any other dictionary, add it with count 0
            dictCheck(word)
                    
    return no_of_story_class,no_of_ask_hn_class,no_of_show_hn_class,no_of_poll_class 

In [82]:
'''
create_model() = build a probabilistic model from the training set.
vocabulary = vocabulary with all the words it contains in Title which is At 2018
story_dictionary = word frequency map for class story
ask_hn_dictionary = word frequency map for class ask_hn
show_hn_dictionary = word frequency map for class show_hn
poll_dictionary = word frequency map for class poll

'''
def create_model(vocabulary,story_dictionary,ask_hn_dictionary,show_hn_dictionary,poll_dictionary,ch):
    

    global story_cp_dictionary    # stores words in story class and their conditional probabilities
    global ask_hn_cp_dictionary   # stores words in ask_hn class and their conditional probabilities
    global show_hn_cp_dictionary # stores words in show_hn class and their conditional probabilities
    global poll_cp_dictionary     # stores words in poll class and their conditional probabilities


    if ch==0:
        f = open("model-2018.txt","w+")    # creating file that would store the model
    elif ch==1:
        f = open("stopword-model.txt","w+")    # creating file that would store the model
            
    N = len(vocabulary)    # getting size of vocabulary
    delta = 0.5    # smoothing value
    
    smoothed_N = (delta * N)
    # calculating smoothed denominator for calculating condinational probability of story class
    story_denominator = sum(story_dictionary.values()) + smoothed_N
    
    # calculating smoothed denominator for calculating condinational probability of ask_hn class
    ask_hn_denominator = sum(ask_hn_dictionary.values()) + smoothed_N
    
    # calculating smoothed denominator for calculating condinational probability of show_hn class
    show_hn_denominator = sum(show_hn_dictionary.values()) + smoothed_N
    
    # calculating smoothed denominator for calculating condinational probability of poll class
    poll_denominator = sum(poll_dictionary.values()) + smoothed_N
    
    for i,word in enumerate(vocabulary):
        
        freq_in_story = story_dictionary[word]                               # frequency of word in story dictionary
        c_p_in_story = (freq_in_story + delta) / story_denominator         # conditional probabiltiy of word in story class
        freq_in_ask_hn = ask_hn_dictionary[word]                             # frequency of word in ask_hn dictionary
        c_p_in_ask_hn = (freq_in_ask_hn + delta) / ask_hn_denominator      # conditional probabiltiy of word in ask_hn class
        freq_in_show_hn = show_hn_dictionary[word]                             # frequency of word in show_hn dictionary
        c_p_in_show_hn = (freq_in_show_hn + delta) / show_hn_denominator   # conditional probabiltiy of word in show_hn class
        freq_in_poll = poll_dictionary[word]                               # frequency of word in poll dictionary
        c_p_in_poll = (freq_in_poll + delta) / poll_denominator            # conditional probabiltiy of word in poll class
        
        story_cp_dictionary[word] = c_p_in_story
        ask_hn_cp_dictionary[word] = c_p_in_ask_hn
        show_hn_cp_dictionary[word] = c_p_in_show_hn
        poll_cp_dictionary[word] = c_p_in_poll
        
        # writing all data to model-2018.txt
        f.write(str(i+1)+'  '+word+'  '+str(freq_in_story)+'  '+str( "{:.8f}".format(float( c_p_in_story )) )+'  '+str(freq_in_ask_hn)+'  '+str( "{:.8f}".format(float( c_p_in_ask_hn )) )+'  '+str(freq_in_show_hn)+'  '+str( "{:.8f}".format(float( c_p_in_show_hn )) )+'  '+str(freq_in_poll)+'  '+str( "{:.8f}".format(float( c_p_in_poll )) )+'\n')
    
    f.close()    # closing the file
    


In [83]:
'''
Run classifier on the 2019 testing dataset and create a single file named baseline-result.txt with your classification results. 

For each test file, baseline-result.txt will contain:
    1. a line counter, followed by 2 spaces
    2. the name of the test post Title, followed by 2 spaces
    3. the classification as given by your classifier (the label story, ask-hn, show-hn or poll), followed by 2 spaces
    4. the score of the class story as given by your classifier, followed by 2 spaces
    5. the score of the class ask-hn as given by your classifier, followed by 2 spaces
    6. the score of the class show-hn as given by your classifier, followed by 2 spaces
    7. the score of the class poll as given by your classifier, followed by 2 spaces
    8. the correct classification of post, followed by 2 spaces
    9. the label right or wrong (depending on the case), followed by a carriage return.

'''
def evaluate_the_model(ch):
    
    global story_dictionary      # stores words in story class and their frequencies
    global ask_hn_dictionary      # stores words in ask_hn class and their frequencies
    global show_hn_dictionary    # stores words in show_hn class and their frequencies
    global poll_dictionary        # stores words in poll class and their frequencies
    global story_cp_dictionary    # stores words in story class and their conditional probabilities
    global ask_hn_cp_dictionary   # stores words in ask_hn class and their conditional probabilities
    global show_hn_cp_dictionary # stores words in show_hn class and their conditional probabilities
    global poll_cp_dictionary     # stores words in poll class and their conditional probabilities
    global vocabulary         # stores all unique words present in dataset
    global word_not_count     # stores all unique removed words present in dataset
    global stopword 
    
    
    # applying log10 on prior probabilities
    if prior_prob_of_story == 0:
        log_of_story=0
    else:
        log_of_story = math.log10(prior_prob_of_story)
    if prior_prob_of_ask_hn == 0:
        log_of_ask_hn=0
    else:
        log_of_ask_hn = math.log10(prior_prob_of_ask_hn)
    if prior_prob_of_show_hn == 0:
        log_of_show_hn=0
    else:
        log_of_show_hn = math.log10(prior_prob_of_show_hn)
    if prior_prob_of_poll == 0:
        log_of_poll=0
    else:
        log_of_poll = math.log10(prior_prob_of_poll)

    score_log_story   =  log_of_story      # score for story class
    score_log_ask_hn  =  log_of_ask_hn     # score for ask_hn class
    score_log_show_hn =  log_of_show_hn    # score for show_hn class
    score_log_poll    =  log_of_poll       # score for poll class

    if ch == 0:
        f = open("baseline-result.txt", "w+")   # 'w+' for reading and writing
    elif ch == 1:
        f = open("stopword-result.txt", "w+")   # 'w+' for reading and writing
        
    f.truncate(0)

    temp_counter=0
    right_classify=0
    wrong_classify=0
    
    for index, row in df_test.iterrows():
        temp_counter=temp_counter+1
        correct_class_type = row['Post Type']
        line = row['Title']
        line = line.strip()                                  # Remove the leading spaces and newline character
        lower_line = str.lower(line)                         # Convert characters in line to lowercase to avoid case mismatch
        valid_words = re.split("[^a-zA-Z]",lower_line)       # filter words following the given regex
        valid_words = list(filter(None, valid_words))        # filter words with length greater than 0
    
        for word in valid_words:
            if word in vocabulary:
                score_log_story = float(score_log_story) + math.log10(story_cp_dictionary[word])        # add log10 of conditional probability of word in story_cp_dictionary
                score_log_ask_hn = float(score_log_ask_hn) + math.log10(ask_hn_cp_dictionary[word])     # add log10 of conditional probability of word in ask_hn_cp_dictionary
                score_log_show_hn = float(score_log_show_hn) + math.log10(show_hn_cp_dictionary[word])  # add log10 of conditional probability of word in show_hn_cp_dictionary
                score_log_poll = float(score_log_poll) + math.log10(poll_cp_dictionary[word])           # add log10 of conditional probability of word in poll_cp_dictionary

        score_list = [score_log_story,score_log_ask_hn,score_log_show_hn,score_log_poll ]
        score_index =  score_list.index(max(score_list))
    
        if score_index == 0:
            predicted_class_type = "story"
        elif score_index == 1:
            predicted_class_type = "ask_hn"
        elif score_index == 2:
            predicted_class_type = "show_hn"
        elif score_index == 3:
            predicted_class_type = "poll"
    
        if(correct_class_type == predicted_class_type):
            label = "right"
            right_classify=right_classify+1
        else:
            label = "wrong"
            wrong_classify=wrong_classify+1
    
        # format scores to appropriate string value
        score_log_story = str( "{:.8f}".format(float(score_log_story)))
        score_log_ask_hn = str( "{:.8f}".format(float(score_log_ask_hn)))
        score_log_show_hn = str( "{:.8f}".format(float(score_log_show_hn)))
        score_log_poll = str( "{:.8f}".format(float(score_log_poll)))
        
        # writing results to result.txt
        f.write(str(str(temp_counter)+"  "+str(line)+"  "+str(predicted_class_type)+"  "+str(score_log_story)+"  "+str(score_log_ask_hn)+"  "+str(score_log_show_hn)+"  "+str(score_log_poll)+"  "+str(correct_class_type)+"  "+str(label)+"\n"))

    f.close()    # closing file
    return right_classify,wrong_classify

# Task 2: Use ML Classifier to test dataset

In [84]:
no_of_story_class,no_of_ask_hn_class,no_of_show_hn_class,no_of_poll_class = word_count_directory(df_train,0)
total_no_of_files = no_of_story_class+no_of_ask_hn_class+no_of_show_hn_class+no_of_poll_class


print(no_of_story_class,no_of_ask_hn_class,no_of_show_hn_class,no_of_poll_class)
# prior probabilities of four class
prior_prob_of_story = no_of_story_class / total_no_of_files
prior_prob_of_ask_hn = no_of_ask_hn_class / total_no_of_files
prior_prob_of_show_hn = no_of_show_hn_class / total_no_of_files
prior_prob_of_poll = no_of_poll_class / total_no_of_files

print(len(vocabulary),len(story_dictionary),len(ask_hn_dictionary),len(show_hn_dictionary),len(poll_dictionary))

vocabulary = sorted(vocabulary)    # sorting the vocabulary to maintain order in model.txt

# put all the removed/invalid words in an file named “remove_word.txt”.
with open('vocabulary.txt', 'w') as f:
    f.writelines(["%s\n" % item  for item in vocabulary])
    
# put all the removed/invalid words in an file named “remove_word.txt”.
with open('remove_word.txt', 'w') as f:
    f.writelines(["%s\n" % item  for item in word_not_count])
    
create_model(vocabulary,story_dictionary,ask_hn_dictionary,show_hn_dictionary,poll_dictionary,0)
right_classify,wrong_classify=evaluate_the_model(0)
print(right_classify,wrong_classify)

<class 'set'>
4572 222 206 0
9544 9544 9544 9544 9544
4606 394


# Task 3: Experiments with the classifier
### Experiment 1: Stop-word Filtering

In [85]:
data_clear()
no_of_story_class,no_of_ask_hn_class,no_of_show_hn_class,no_of_poll_class = word_count_directory(df_train,1)
total_no_of_files = no_of_story_class+no_of_ask_hn_class+no_of_show_hn_class+no_of_poll_class

print(type(vocabulary))

# prior probabilities of four class
prior_prob_of_story = no_of_story_class / total_no_of_files
prior_prob_of_ask_hn = no_of_ask_hn_class / total_no_of_files
prior_prob_of_show_hn = no_of_show_hn_class / total_no_of_files
prior_prob_of_poll = no_of_poll_class / total_no_of_files

print(len(vocabulary),len(story_dictionary),len(ask_hn_dictionary),len(show_hn_dictionary),len(poll_dictionary))

vocabulary = sorted(vocabulary)   # sorting the vocabulary to maintain order in model.txt


# put all the removed/invalid words in an file named “remove_word.txt”.
with open('stopword-vocabulary.txt', 'w') as f:
    f.writelines(["%s\n" % item  for item in vocabulary])
    
# put all the removed/invalid words in an file named “remove_word.txt”.
with open('stopword-remove_word.txt', 'w') as f:
    f.writelines(["%s\n" % item  for item in word_not_count])
    
create_model(vocabulary,story_dictionary,ask_hn_dictionary,show_hn_dictionary,poll_dictionary,1)
right_classify,wrong_classify=evaluate_the_model(1)
print(right_classify,wrong_classify)

<class 'set'>
<class 'set'>
9405 9405 9405 9405 9405
4569 431


In [86]:
<class 'set'>
0 9405 9405 9405 9405

SyntaxError: invalid syntax (<ipython-input-86-76a2fe70f0cf>, line 1)